In [1]:
import tools
from tools.imports import *
from tools.document_tools import GoogleDocWriteTool
from tools.file_mgmt_tools import MoveFileTool, CreateFolderTool, FolderMovementTool, FileOrganizerTool
from tools.miscellaneous_mgmt import GoogleDriveUploadTool, GmailSendPdfTool, GoogleSheetsUpdateTool
from tools.initialize_groq import init_groq

In [2]:
client, llm = init_groq()

In [3]:
credentials_path = "C:\\DEV\\WebdevFolder\\client_secret_291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com.json"

In [4]:
my_tools = []


my_tools.append(
    
    GoogleDocWriteTool()
)

my_tools.append(
    
    GoogleDriveUploadTool()
)

my_tools.append(
    
    GoogleSheetsUpdateTool()
)
credentials_path = "C:\\DEV\\WebdevFolder\\client_secret_291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com.json"
my_tools.append(
    GmailSendPdfTool(credentials_path)
)

my_tools.append(
    MoveFileTool(credentials_path)
)

my_tools.append(
    CreateFolderTool(credentials_path)
)

my_tools.append(
    FolderMovementTool(credentials_path)
)

my_tools.append(
    FileOrganizerTool(credentials_path)
)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A60417%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=PsL3s8DzQOmLNBYAZJ2Nl7sdEPqKob&access_type=offline
Authenticating with Google Drive API...
Running local server for authentication...
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A60426%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=7Ss0fgLTW6uwAkhAirlB3oXHbQq0e6&access_type=offline
Authentication successful.
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.

In [7]:
import tools.initialize_groq
import random
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, AgentType
from google.cloud import texttospeech

# Define the template
template = """
You are a nice, great assistant. User will tell you things. You just respond.
YOU SHALL NOT INDICATE ANY TOOL USE UNTIL YOU KNOW YOU HAVE EVERYTHING YOU NEED.
DO NOT ASSUME USER WANTS TO DO ANYTHING AT ALL UNLESS YOU ARE 100% SURE!!!!! UNDERSTAND??????!!!!!! OR ELSE I WILL BECOME ANGRY

You will need credentials of the user. Here they are:
Name: {credentials[name]}
Email: {credentials[email]}
Recipient Email: {credentials[recemail]}
Phone: {credentials[phone]}
-------
You may need the path to any documents. Here it is:
{thepath}

Here are the tools you can use. DO NOT TRY TO USE ANYTHING OTHER THAN THESE:
{tools}

HERE IS GUIDANCE ON EXACTLY WHAT TO PASS IN:
GoogleDocWriteTool: INPUT EVERYTHING CORRECTLY EVEN IF ITS LONG!!!!!!!! INPUT ONLY WHAT YOURE GIVEN ! NO CREATIVITIES!!!! append-->true or false lowercase only!!! NO DICTIONARIES, JUST EVERYTHING PASSED AS IT ISSS!!!!!--> INPUT_TEXT: generate it yourself | APPEND: decide whether or not you want to append to document (must be in lowercase)
MoveFileTool: INPUT DIRECTLY THE FOLLOWING INPUTS --> FILE_NAME please deduce from the response and then pass it into tool: {input} ; FOLDER_NAME deduce from the response and then pass it into tool - PASS IN 'root' if user says move into google drive or my drive: {input}
GoogleSheetsUpdateTool: Input ONLY ONLY these into the tool PROPERLY! --> {credentials[name]}, {credentials[phone]}
CreateFolderTool: INPUT DIRECTLY THE FOLLOWING INPUTS --> FOLDER NAME: the folder name as specified in response --> {input} ||| PLEASE OUTPUT THE EXACT NAME OF THE FOLDER THAT WAS CREATED
MoveFolderTool: INPUT DIRECTLY THE FOLLOWING INPUTS --> FOLDER_NAME please deduce from the response and then pass it into tool: {input} ; NEW_PARENT_FOLDER_NAME also deduce from response - PASS IN 'root' if user says move into google drive or my drive: {input}
FileOrganizationTool: INPUT DIRECTLY THE FOLLOWING INPUTS --> PARENT_FOLDER_NAME please deduce from the response and then pass it into tool: {input}
GoogleDriveUploadTool: INPUTS --> FILE_PATH: {thepath} USER_EMAIL: {credentials[email]}
GmailSendPdfTool: IF THERE IS A FILE TO SEND, INPUT DIRECTLY THE FOLLOWING INPUTS --> SENDER EMAIL: {credentials[email]}, RECIPIENT EMAIL: {credentials[recemail]}, SUBJECT: you generate it yourself, BODY: you generate it yourself and it has to be an informative body, PDF PATH: {thepath}
GmailSendPdfTool: IF THERE IS NO FILE TO SEND, INPUT DIRECTLY THE FOLLOWING INPUTS --> SENDER EMAIL: {credentials[email]}, RECIPIENT EMAIL: {credentials[recemail]}, SUBJECT: you generate it yourself, BODY: you generate it yourself and it has to be an informative body, PDF PATH: nothing

If what the user says is one of these you must explicitly say AT THE END OF YOUR RESPONSE in this very format depending on which tool - "I will use the [google doc write tool, gmail tool, sheets tool, move file to a folder tool, create folder tool, the move folder tool, file organization tool, OR drive upload tool]". 
so that user can confirm if you got it correctly. 
When writing to a file, just start ONLY ONLY ONLY THE BEGINNING OF THE ENTIRE CONTENT with '@' symbol when generating file content. There should only be ONNNEEEEE '@' symbol in response and that is at the beginning!!!!!
WHEN WRITING TO FILE--> If user indicates that they want to add on to existing content, \
you must say 'yes append' before the aforementioned '@' symbol. OTHERWISE YOU SAY 'no append' before the aforementioned '@' symbol

IMPORTANT: IF YOU REMEMBER YOU HAVE USED A TOOL ALREADY (REFER TO CHAT HISTORY), AND YOU ARE IN THE PROCESS OF USING IT AGAIN, \
YOU MUST ASK USER IF THEY ARE SURE TO USE THE TOOL AGAIN OR NOT.

If you have to use a google related tool, you MUST prompt the user or remind user to enter credentials in the doc manager extension. 
If user tells you to do something that is not one of these, you kindly say that you don't have access to that functionality.

---------
Chat History: {chat_history}
User Input: {input}
"""

# Define the prompt template
prompt_template = PromptTemplate(
    input_variables=["tools", "input", "chat_history", "credentials", "thepath"],
    template=template
)

# Initialize the agent
llm.groq_api_key = random.choice(tools.initialize_groq.api_keys)
agent = initialize_agent(
    tools=my_tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    prompt_template=prompt_template,
    handle_parsing_errors=True,
    verbose=True,
)

# Chat history tracking
chat_history = []

# User credentials
credentials = {
    "name": "Sriram Nallani",
    "email": "sriramnallani35@gmail.com",
    "recemail": "thegreatchrisjohn@gmail.com",
    "phone": "4973993399"
}

# Google TTS client initialization
def synthesize_speech(text, output_file):
    client = texttospeech.TextToSpeechClient()
    input_text = texttospeech.SynthesisInput(text=text)
    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US",
        ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL
    )
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )
    response = client.synthesize_speech(
        input=input_text,
        voice=voice,
        audio_config=audio_config
    )
    with open(output_file, "wb") as out:
        out.write(response.audio_content)
    print(f'Audio content written to "{output_file}"')

# Define a function to run the agent and handle the response
def run_agent(input_text):
    global chat_history
    tools_str = '\n'.join([f"{tool.name}: {tool.description}" for tool in my_tools])
    response = agent.invoke({
        "input": input_text,
        "chat_history": chat_history,
        "tools": tools_str,
        "credentials": credentials,
        "thepath": ".documents/MyDocument.pdf"
    })
    chat_history.append({"input": input_text, "response": response})
    
    # Use Google TTS to convert the response to speech
    tts_output_file = "output.mp3"
    synthesize_speech(response, tts_output_file)
    print(f'Response audio saved to {tts_output_file}')
    
    return response

# Example user input
user_input = """Can you please write a 5 paragraph essay about laptops and their relation to artificial intelligence, 
and then move the document to a folder called FCPS google. 
Then create a folder named 'dummy folder' and then move the folder called 'destination folder' to this 'dummy folder'. 
Then upload the documents called MyDocument.pdf and 'file_id_history.txt' to google drive please!"""

# Run the agent with the example input
result = run_agent(user_input)
print(result)




> Entering new AgentExecutor chain...
Here is the response:

Action:
```
{
  "action": "GoogleDocWriteTool",
  "action_input": {
    "input_text": "Laptops and Artificial Intelligence

Laptops have revolutionized the way we live and work. With the advent of artificial intelligence, laptops have become even more powerful tools. AI-powered laptops can learn and adapt to our habits, making our lives easier and more efficient.

One of the most significant impacts of AI on laptops is the improvement in processing power. AI algorithms can process vast amounts of data quickly and accurately, making laptops faster and more reliable. This has led to the development of more sophisticated AI-powered applications, such as virtual assistants and predictive analytics tools.

The integration of AI in laptops has also enhanced security features. AI-powered laptops can detect and respond to cyber threats in real-time, providing an additional layer of protection for users. Furthermore, AI-powered faci